In [1]:
import numpy as np; np.random.seed(0)
import pandas as pd
from collections import deque

In [2]:
target_index_size = 10  # 포트폴리오 자산 갯수
n_days_train = 21*6    # 6개월 트래인 기간
n_days_test = 21       # 1개월 테스트 기간
batch_size = 1000      # 한번 트래인시 필요한 데이터 갯수

In [3]:
tickers = pd.read_csv('../data/sp500/tickers.csv', usecols=[1])
n_assets = tickers.shape[0]
print(f'총 자산 수 : {n_assets}')
print('티커 : {}'.format(list(tickers.iloc[:,0])))

총 자산 수 : 505
티커 : ['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALXN', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'AIV', 'AAPL', 'AMAT', 'APTV', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC', 'BK', 'BAX', 'BDX', 'BRK-B', 'BBY', 'BIO', 'BIIB', 'BLK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BF-B', 'CHRW', 'COG', 'CDNS', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CTL', 'CERN', 'CF', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'CXO', 'COP', 'ED', 'STZ', 'COO', 'CPRT', 'GLW', 'CTVA', 'COST', 'COTY', 'CCI', 'CSX', 'CMI', 'CVS', 'DHI', 

In [4]:
target_index = list(np.random.choice(np.arange(n_assets), target_index_size, replace=False))
train_index = []
for i in range(n_assets-1):
    if i not in target_index:
        train_index.append(i)
        
tickers_target = list(tickers.iloc[target_index, 0]) 
tickers_train = list(tickers.iloc[train_index, 0])
        
print('타겟 자산 수 : {}'.format(len(target_index)))
print('타겟 자산 인덱스 : {}'.format(target_index))
print('타겟 티커 : {}'.format(tickers_target), end='\n\n')

print('트래인 자산 수 : {}'.format(len(train_index)))
print('트래인 자산 인덱스 : {}'.format(train_index))
print('트래인 티커 : {}'.format(tickers_train))

타겟 자산 수 : 10
타겟 자산 인덱스 : [90, 97, 477, 340, 395, 15, 318, 399, 159, 153]
타겟 티커 : ['CBOE', 'CERN', 'WRB', 'NLSN', 'O', 'ALB', 'MU', 'RSG', 'DD', 'DPZ']

트래인 자산 수 : 494
트래인 자산 인덱스 : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 91, 92, 93, 94, 95, 96, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 154, 155, 156, 157, 158, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 1

In [5]:
df_pct_change = pd.read_csv('../data/sp500/Pct_Change.csv', index_col='Date', parse_dates=True)
n_days = df_pct_change.shape[0] 
print(df_pct_change.shape)

(14765, 505)


In [6]:
def give_me_x_y():
    
    x = []
    y = []

    for i in range(batch_size):

        while True: 

    #         target_index_size = 20 # 포트폴리오 자산 갯수
    #         n_days_train = 21*6    # 6개월 트래인 기간
    #         n_days_test = 21       # 1개월 테스트 기간
    #         batch_size = 1000      # 한번 트래인시 필요한 데이터 갯수
            start_day_x = np.random.choice(np.arange(n_days-n_days_train-n_days_test), 1)[0] 
            end_day_x = start_day_x + n_days_train - 1 
            start_day_y = start_day_x + n_days_train
            end_day_y = start_day_x + n_days_train + n_days_test - 1 

            asset_index = list(np.random.choice(train_index, target_index_size, replace=False))

            data_x = df_pct_change.iloc[start_day_x:end_day_x+1,asset_index]
            data_y = df_pct_change.iloc[start_day_y:end_day_y+1,asset_index]

            if not data_x.isnull().any().any():
                if not data_y.isnull().any().any():
                    try:
                        x.append(data_x.values.reshape((-1,)))
                        y.append(data_y.values.reshape((-1,)))
                        break
                    except:
                        pass

    x = np.array(x)
    y = np.array(y)
    print(f'트래인 셋 인풋 사이즈 : {x.shape}')
    print(f'트래인 셋 아웃풋 사이즈 : {y.shape}')
    
    return x, y

In [7]:
x, y = give_me_x_y() 
print(f'첫번째 트래인 셋 인풋 : {x[0]}')
print(f'첫번째 트래인 셋 아웃풋 : {y[0]}')

트래인 셋 인풋 사이즈 : (1000, 1260)
트래인 셋 아웃풋 사이즈 : (1000, 210)
첫번째 트래인 셋 인풋 : [ 0.00330993  0.00727584 -0.0041795  ...  0.01477833  0.03571429
  0.00130399]
첫번째 트래인 셋 아웃풋 : [-9.27459937e-03 -1.82778229e-03 -1.05923044e-02  1.22125989e-02
  2.28937729e-03  7.28818706e-03  1.62187210e-03  8.46047157e-03
 -4.48275862e-01  8.46491942e-03 -2.81311537e-04 -2.03458800e-04
  0.00000000e+00  5.72483841e-03 -5.25354043e-03  1.74856798e-02
  1.65008867e-02  7.28923119e-03 -8.12500000e-01  5.48829701e-03
 -2.62631316e-03  6.10500611e-03  1.48568932e-02  1.46896805e-03
  6.88863375e-04  2.07407407e-03  1.22126982e-02  1.13326051e-02
  0.00000000e+00  2.18333601e-02  8.30721003e-04  1.82038835e-03
 -1.82992465e-02 -7.39518396e-03 -3.44194585e-03 -2.06978119e-03
 -2.69784173e-03  1.75509653e-03  0.00000000e+00  2.51374705e-03
  1.43297887e-02 -4.84554815e-03 -2.01754386e-02  2.33975740e-03
  8.28920101e-03 -2.37037037e-03 -5.25999399e-03  4.71698113e-03
  4.33333333e+00 -8.61933866e-03 -7.85881917e-03  8.52

In [8]:
for i in range(3):
    give_me_x_y() 

트래인 셋 인풋 사이즈 : (1000, 1260)
트래인 셋 아웃풋 사이즈 : (1000, 210)
트래인 셋 인풋 사이즈 : (1000, 1260)
트래인 셋 아웃풋 사이즈 : (1000, 210)
트래인 셋 인풋 사이즈 : (1000, 1260)
트래인 셋 아웃풋 사이즈 : (1000, 210)


In [10]:
buffer = deque(maxlen=3)
for i in range(10):
    buffer.append(give_me_x_y())
    #print(buffer)
    print(len(list(buffer)))

트래인 셋 인풋 사이즈 : (1000, 126)
트래인 셋 아웃풋 사이즈 : (1000, 21)
1
트래인 셋 인풋 사이즈 : (1000, 126)
트래인 셋 아웃풋 사이즈 : (1000, 21)
2
트래인 셋 인풋 사이즈 : (1000, 126)
트래인 셋 아웃풋 사이즈 : (1000, 21)
3
트래인 셋 인풋 사이즈 : (1000, 126)
트래인 셋 아웃풋 사이즈 : (1000, 21)
3
트래인 셋 인풋 사이즈 : (1000, 126)
트래인 셋 아웃풋 사이즈 : (1000, 21)
3
트래인 셋 인풋 사이즈 : (1000, 126)
트래인 셋 아웃풋 사이즈 : (1000, 21)
3
트래인 셋 인풋 사이즈 : (1000, 126)
트래인 셋 아웃풋 사이즈 : (1000, 21)
3
트래인 셋 인풋 사이즈 : (1000, 126)
트래인 셋 아웃풋 사이즈 : (1000, 21)
3
트래인 셋 인풋 사이즈 : (1000, 126)
트래인 셋 아웃풋 사이즈 : (1000, 21)
3
트래인 셋 인풋 사이즈 : (1000, 126)
트래인 셋 아웃풋 사이즈 : (1000, 21)
3
